\newcommand{\d}{\,\mathrm{d}}
\newcommand{\e}{\mathrm{e}}
\newcommand{\E}{\mathbb{E}}



# Ito Processes and Ito's Formula {#sec-c:ito}

We use the changes of a Brownian motion to model randomness.  We build other stochastic processes using those changes.  The general idea is
$$\text{change} = \text{mean} + \text{std dev} \times \text{change in Brownian motion}\,.$$
The mathematical foundations for our construction were created by K. Ito.  The key concepts are the Ito integral, Ito processes, and Ito's formula (also called Ito's lemma).  Using these foundations, we can build quite general processes from changes in Brownian motions, including processes with non-normal distributions.

## Examples {#sec-s:discreteito}

We begin with some simple examples.  Consider a discrete partition of a time interval:
$$0=t_0 < t_1< \cdots < t_{n-1} < t_n=t$$
with equally spaced times.  Let $\Delta t$ denote the difference between successive times.  

First, let's drop the randomness entirely.  Consider the equation
$$X_{t_i} -  X_{t_{i-1}} = \mu X_{t_{i-1}}\Delta t$$ {#eq-ito_discrete_1}
for a constant $\mu$.  Thus, we have "change $\,=\,$ mean," where the mean is proportional to the previous value with proportionality factor $\mu \Delta t$.  @fig-discrete-interest presents a plot of $X$, for particular values of $X_0$, $\mu$, and $\Delta t$.

If we increase $n$, making $\Delta t$ smaller, then $X$ converges to the solution of the ordinary differential equation
$$\d X_t = \mu X_t\d  t\,.$$ {#eq-ito_discrete_2}
@eq-ito_discrete_2 has a known solution, which is 
$$X_t = X_0 \e^{\mu t}\,.$$ {#eq-ito_discrete_3}
To verify this, we only need to differentiate $X$ defined in @eq-ito_discrete_3:
$$\frac{\d X_t}{\d t} = \mu X_0 \e^{\mu t} = \mu X_t\,.$$
The function presented in @eq-ito_discrete_3 is also shown in @fig-discrete-interest.

::: Extra

To see how one might guess that @eq-ito_discrete_3 is the solution of @eq-ito_discrete_2, we can examine the logarithm of $X$.  A general rule gives us $\d \log X = \d X / X$, so $\d \log X = \mu \d t$.  We can integrate both sides of this to obtain $\log X_t - \log X_0 = \mu t$.  Now, rearranging and exponentiating gives $X_t = X_0\e^{\mu t}$.  Later, we follow similar steps to see that @eq-ito_discrete_6 is the solution of @eq-ito_discrete_5.
:::


In [ ]:
#| label: fig-discrete-interest
#| fig-cap: The functions $X$ satisfying @eq-ito_discrete_1 (difference equation) and @eq-ito_discrete_3 (differential equation) for X0 = 1, mu = 1, and Delta t = 0.1.

import numpy as np
import plotly.graph_objects as go

mu = 1
n = 10   # number of subdivisions
t = 1     # last date
Deltat = t/n
X1 = np.ones(n+1)
for i in range(1, n+1):
    X1[i] = X1[i-1] + mu * X1[i-1] * Deltat

X2 = np.exp(mu * np.arange(0, t+Deltat, Deltat))

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.arange(0, t+Deltat, Deltat), 
        y=X1, 
        mode='markers', 
        name='Difference Eq.',
        hovertemplate='t = %{x:.2f}<br>B = %{y:.2f}<extra></extra>'  # 
    )
)

fig.add_trace(
    go.Scatter(
        x=np.arange(0, t+Deltat, Deltat), 
        y=X2, 
        mode='lines', 
        name='Differential Eq.',
        hovertemplate='t = %{x:.2f}<br>B = %{y:.2f}<extra></extra>'  # 
    )
)
fig.update_layout(
    showlegend=True,
    xaxis_title='Time',
    yaxis_title='',
    template='plotly_white',
    height=300,
    legend=dict(
        x=0.1,
        y=1,
        xanchor="left",
        yanchor="top",
    )
)

fig.show()

Now, let's include randomness.  Let's make the noise proportional to the value of $X$,  So, let $B$ be a standard Brownian motion and consider the equation
$$X_{t_i} - X_{t_{i-1}} = \mu X_{t_{i-1}}\Delta t + \sigma X_{t_{i-1}} \Delta B_{t_i}$$ {#eq-ito_discrete_4}
where $\sigma$ is another constant, and $\Delta B_{t_i} = B_{t_i} - B_{t_{i-1}}$.  A solution $X$ of this equation  has random paths, due to the random noise $\Delta B_{t_i}$.  An example of a path is shown in @fig-discrete-random.
Ito showed how we can take the limit of this equation as we make $\Delta t$ smaller and make sense of the equation
$$
\d X_t = \mu X_t\d  t + \sigma X_t \d  B_t\,.
$$ {#eq-ito_discrete_5}
The solution $X$ of @eq-ito_discrete_5 is
$$X_t = \e^{(\mu  - \sigma^2/2)t + \sigma B_t}\,.$$ {#eq-ito_discrete_6}
We can show that $X$ defined in @eq-ito_discrete_6 satisfies @eq-ito_discrete_5 by differentiating, as we showed that $X$ defined in @eq-ito_discrete_3 satisfies @eq-ito_discrete_2.  However, we first need to explain Ito's formula, which is a formula for differentiating functions of Brownian motions and, more generally, functions of Ito processes.  An approximate path of $X$ is shown in @fig-discrete-random.  It is generated by taking $\Delta t$ very small, just as we generated approximate paths of Brownian motions in @sec-c:brownian.  


In [ ]:
#| label: fig-discrete-random
#| fig-cap: Paths of the processes satisfying @eq-ito_discrete_4 (difference equation) and @eq-ito_discrete_6 (differential equation) for X0 = 1, mu = 1, Delta t = 0.1, and sigma = 1.

import numpy as np
import plotly.graph_objects as go

mu = 1
sigma = 1
t = 1

# Brownian path
n = 1000   
dt = t/n
dB = np.random.normal(scale = np.sqrt(dt), size=n)
B = np.zeros(n+1)
B[1:] = np.cumsum(dB)

# Brownian path with discrete steps
n_discrete = 10
Deltat = t/n_discrete
B_discrete = B[::int(n/n_discrete)]
DeltaB = np.diff(B_discrete)

# X in discrete-time 
X1 = np.ones(n_discrete+1)
for i in range(1, n_discrete+1):
    X1[i] = X1[i-1] + mu * X1[i-1] * Deltat + sigma * X1[i-1] * DeltaB[i-1]

# Continuous-time
X2 = np.exp((mu - 0.5 * sigma**2) * np.arange(0, t+dt, dt) + sigma * B)

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.arange(0, t+Deltat, Deltat), 
        y=X1, 
        mode='markers', 
        name='Difference Eq.',
        hovertemplate='t = %{x:.2f}<br>B = %{y:.2f}<extra></extra>'  # 
    )
)

fig.add_trace(
    go.Scatter(
        x=np.arange(0, t+dt, dt), 
        y=X2, 
        mode='lines', 
        name='Differential Eq.',
        hovertemplate='t = %{x:.2f}<br>B = %{y:.2f}<extra></extra>'  # 
    )
)
fig.update_layout(
    showlegend=True,
    xaxis_title='Time',
    yaxis_title='',
    template='plotly_white',
    height=300,
    legend=dict(
        x=0.1,
        y=1,
        xanchor="left",
        yanchor="top",
    )
)

fig.show()

The functions and processes $X$ defined in this section have important interpretations.  @eq-ito_discrete_1 can be rewritten to say that the percent change in $X$ is $\mu \Delta t$.  This could represent the value of a savings account that earns interest of $\mu \Delta t$ in each period of length $\Delta t$.  This is the common way of calculating, for example, monthly interest, where $\mu$ is called the annual rate of interest and $\Delta t$ would be $1/12$.  The limiting @eq-ito_discrete_3 is called continuous compounding of interest.

Similarly, @eq-ito_discrete_4 can be rewritten to say that the percent change in $X$ is $\mu \Delta t + \sigma \Delta B$.  This represents a random rate of return -- for example, the return of a stock.  The expected rate of return in this case is $\mu \Delta t$, and the variance of the rate of return is $\sigma^2 \Delta t$.  The limiting @eq-ito_discrete_3 is called continuous compounding of returns.



## Ito Processes {#sec-s:itoprocesses}

The meaning of @eq-ito_discrete_2 is that, for all $t  > 0$,
$$X_t = X_0 +  \int_0^t \mu X_s \d s\,.$$
We assume the reader is familiar with integrals, so we do not explain this further.  The function of time $X$ defined in @eq-ito_discrete_3 satisfies this equation.  Similarly, the meaning of @eq-ito_discrete_5 is that, for all $t > 0$,
$$X_t = X_0 +  \int_0^t \mu X_s \d s + \int_0^t \sigma X_s \d B_s\,.$$
The first integral in this formula is an ordinary integral.  The second is an Ito integral, which is to be explained.  The sum of an ordinary integral and an Ito integral is called an Ito process.  An Ito process always has continuous paths. 

Let's depart from the example of the previous section and consider a process $X$ satisfying, for all $t>0$,
$$X_t = \int_0^t \alpha_s\d s + \int_0^t \theta_s \d B_s\,.$${#eq-ito1}
where $\alpha$ and $\theta$ can be stochastic processes.  The example of the previous section fits this form, because we could take $\alpha_s = \mu X_s$ and $\theta_s = \sigma X_s$.  The definition of the Ito integral
$$\int_0^t \theta_s \d B_s$$
is relatively complicated.  It is enough for our purposes to know that it can be approximated by a  discrete sum
$$\sum_{i=1}^n \theta_{t_{i-1}}(B_{t_i} - B_{t_{i-1}})\,,$$
given a partition
$$ 0 = t_0 < \cdots < t_n = t\,,$$
when $n$ is large and the time between successive dates is small.  The Ito integral exists provided $\theta$ does not anticipate the future (so $\theta_{t_{i-1}}$ is independent of the increment $B_{t_i}- B_{t_{i-1}}$) and provided $\theta$ does not explode to $\pm \infty$ in finite time, so 
$$\int_0^t \theta_s^2 \d s < \infty$$
for all $t > 0$, with probability one.

We also write @eq-ito1 as
$$
\d X_t = \alpha_t\d t + \theta_t \d B_t\,. 
$${#eq-ito2}
We interpret this as "change $=$ mean $+$ noise" with $\alpha_t \d t$ being the mean and $\theta_t\d B_t$ being the mean-zero random noise.    The quantity $\alpha_t$  is also called the drift of the process $X$ at time $t$.   \index{drift} The coefficient $\theta_t$ is called the diffusion coefficient of $X$ at time $t$.  If $\alpha$ and $\theta$ are constant, it is standard to refer to an Ito process $X$ as a $(\alpha,\theta)$--Brownian motion.  When they are constant, we obtain
$$X_t= X_0 + \alpha t + \theta B_t\,.$$

An Ito process as in @eq-ito1 can be a martingale only if $\alpha=0$.  This should seem sensible, because $\alpha\d  t$ is the expected change in $X$, and a process is a martingale only if its expected change is zero.   This observation plays a fundamental role in deriving asset pricing formulas.  Conversely, if $\alpha=0$ and 
$$
\E \left[\int_0^t \theta^2_s\d   s\right] < \infty
$$ {#eq-regularity1}
for each $t$, then the Ito process is a continuous martingale, and the variance of its date--$t$ value, calculated with the information available at date $0$, is:
$$\mathrm{var}(X_t) = \E \left[\int_0^t \theta^2_t\d  s\right]\; .$$  

## Quadratic and Joint Variation of  Ito Processes


To compute the quadratic variation of an Ito process, we use the following simple and important rules (for the sake of brevity, we drop the subscript $t$ from $B_t$ here and sometimes later).  These rules should be regarded as mnemonic devices.  The calculations we do with them lead to the correct results, but the objects have no real mathematical meaning.

::: Principle

$$
(\d  t)^2 = 0\;, 
$$ {#eq-dtsquared}

$$
(\d  t)(\d  B) =0\;, 
$$ {#eq-dB}

$$
(\d  B)^2 =\d  t\;. 
$$ {#eq-dBsquared}



:::


We apply these rules to compute the quadratic variation of any Ito proces $X$ as follows:

::: Principle

If $\d  X = \alpha\d   t + \theta\d   B$ for a Brownian motion $B$, then
\begin{align}
(\d  X)^2 &= (\alpha\d t+\theta\d   B)^2\\
&= \alpha^2(\d  t)^2 + 2\alpha\theta(\d  t)(\d  B) + \theta^2(\d  B)^2\\
&= \theta^2\d   t\;.
\end{align}
To compute the quadratic variation of the Ito process $X$ over any particular period of time, we integrate $(\d X)^2$ over that period as^[In a more formal mathematical presentation, one normally writes $\d \langle X,X\rangle$ for what we are writing here as $(\d  X)^2$.  This is the differential of the quadratic variation process, and the quadratic variation through date $t$ is
$$
\langle X,X\rangle _t = \int_0^t \d \langle X,X\rangle_s = \int_0^t \sigma^2_s\d   s\;.
$$
Our mnenomic device of squaring differentials leads us to the correct formula.]]
$$
\int_0^t (\d  X_s)^2 = \int_0^t \theta^2_s\d   s\;.
$$ {#eq-itoquadraticvariation2}
:::


Now consider two Ito processes:
$$
\d  X_{1t} = \mu_{1t}\d   t + \sigma_{1t}\d   B_{1t}\;,
$$ {#eq-itoprocess110}

$$
\d  X_{2t} = \mu_{2t}\d   t + \sigma_{2t}\d   B_{2t}\;,
$$ {#eq-itoprocess111}

where $B_1$ and $B_2$ are standard Brownian motions.  We calculate the product of differentials of Ito processes as follows.

::: Principle

If $X_1$ and $X_2$ are Ito processes as in @eq-itoprocess110 and @eq-itoprocess111, then
$$(\d X_1)(\d X_2) = (\sigma_{1}\d B_{1})(\sigma_{2}\d B_{2})= \sigma_1\sigma_2\rho\d t$${#eq-products-differentials}
where $\rho$ is the correlation process of the two Brownian motions. 
:::

The real meaning of this rule is that it is possible to calculate the joint variation (i.e., limit of sum of products of changes) of the two Ito processes from $0$ to $t$ as
$$\int_0^t (\d X_{1s})(\d X_{2s}) = \int_0^t (\sigma_{1s}\d B_{1s})(\sigma_{2s}\d B_{2s}) = \int_0^t \sigma_{1s}\sigma_{2s}\rho_s\d s\,.
$$ {#eq-itojointvariation}
The last integral in this equation is the correct formula for the quadratic variation.  As with squaring differentials, taking products of differentials is a mnemonic device to get us to the correct formula.^[A somewhat more precise definition than our previous description of the stochastic integral $\int_0^t \sigma_{1,t} dB_{1t}$ is when @eq-regularity1 holds, the stochastic integral is the (unique) martingale with joint variation with any other Ito process given by @eq-itojointvariation.]

## Introduction to Ito's Formula {#sec-s:itosformula}

First we recall some facts of the ordinary calculus.  If $y=f(x)$ and $x_t = g(t)$ with $f$ and $g$ being continuously differentiable functions, then 
$$\frac{\d  y}{\d  t} = \frac{\d  y}{\d  x}\times \frac{\d  x}{\d  t} = f'(x_t)g'(t)\; .$$
This implies that, for each $t>0$,
$$y_t = f(x_t) = y_0 + \int_0^t \frac{\d  y}{\d  s}\d   s = y_0 + \int_0^t f'(x_s)g'(s)\d   s\; .$$
Substituting $\d x_s = g'(s)\d   s$, we can also write this as
$$
y_t = f(x_t) = y_0 + \int_0^t f'(x_s)\d   x_s\,,
$$ {#eq-ordinarycalculus}
or, in differential form,
$$
dy_t = f'(x_t)\d x_t \,.
$$ {#eq-ordinarycalculus2}
What people frequently remember about integrals from their calculus courses is that there are a lot of tricky substitutions that can be made to simplify the calculation of various integrals.  We won't need those in this book.  All we will use are equations of the form of @eq-ordinarycalculus, which is a special case of the Fundamental Theorem of Calculus, which says that a function is the integral of its derivative.  Intuitively, we can think of @eq-ordinarycalculus as saying that the change in $y$ over a discrete interval (from $0$ to $t$) is the continuous sum (integral) of its infinitesimal changes.  

We will contrast  @eq-ordinarycalculus  with the following special case of Ito's formula for the calculus of Ito processes.  

::: Principle 
If $B$ is a Brownian motion and $Y = f(B)$ for a twice-continuously differentiable function $f$, then 
$$\d   Y_t = f'(B_t)\d   B_t + \frac{1}{2}f''(B_t)\d   t  \,. 
$$ {#eq-ito0}
:::
Comparing @eq-ito0 to @eq-ordinarycalculus2, we see that Ito's formula has an extra term involving the second derivative $f''$.   

@eq-ito0 implies that $Y=f(B)$ is an Ito process with drift $f''(B_t)/2$ and diffusion coefficient $f'(B_t)$.  The real meaning of @eq-ito0 is the integrated form:
$$
Y_t = f(B_t) = Y_0 + \int_0^t f'(B_s)\d   B_s + \frac{1}{2}\int_0^t f''(B_s)\d   s\;.
$$ {#eq-itonew1}
Thus, the change in $Y$ over a discrete interval is again the continuous sum of its infinitesimal changes, but now the infinitesimal changes are given by @eq-ito0.  Note that the first integral in @eq-itonew1 is an Ito integral.

To gain some intuition for the extra term in Ito's formula, we return to the ordinary calculus.  Given dates $s<t$, the derivative defines a linear approximation of the change in $y$ from $s$ to $t$; that is, setting $\Delta x = x_t-x_s$ and $\Delta y = y_t - y_s$, we have the approximation
$$\Delta y \approx f'(x_s) \,\Delta x\; .$$
A better approximation is given by the second-order Taylor series expansion
$$\Delta y \approx f'(x_s)\,\Delta x + \frac{1}{2} f''(x_s)\,(\Delta x)^2\; .$$
An interpretation of  @eq-ordinarycalculus is that the linear approximation works perfectly for infinitesimal time periods $\d  s$, because we can compute the change in $y$ over the time interval $[0,t]$ by summing up the infinitesimal changes $f'(x_s)\d   x_s$.  In other words, the second-order term $\frac{1}{2} f''(x_s)\,(\Delta x)^2$ vanishes when we consider very short time periods.

The second-order Taylor series expansion in the case of $Y=f(B)$ is 
$$\Delta Y \approx f'(B_s)\,\Delta B + \frac{1}{2} f''(B_s)\,(\Delta B)^2\; .$$
For example, given a partition $0=t_0 < t_1 < \cdots < t_n=t$ of the time interval $[0,t]$, we have, with the same notation we have used earlier,

$$
Y_t = Y_0 + \sum_{i=1}^n \Delta Y_{t_i}  
\approx Y_0 + \sum_{i=1}^n f'(B_{t_{i-1}})\,\Delta B_{t_i} + \frac{1}{2} \sum_{i=1}^n f''(B_{t_{i-1}})\,(\Delta B_{t_i})^2\;.
$$ {#eq-itonew2}


If we make the time intervals $t_i-t_{i-1}$ shorter, letting $n \rightarrow \infty$, then we cannot expect that the extra term here will disappear, leading to the result  of the ordinary calculus shown in @eq-ordinarycalculus, because we know that
$$\lim_{n \rightarrow \infty} \sum_{i=1}^n (\Delta B_{t_i})^2 = t\; ,$$
whereas for the continuously differentiable function $x_t = g(t)$, the same limit is zero.  In fact it seems sensible to interpret the limit of $(\Delta B)^2$ as $(\d  B)^2 =\d  t$.
This is perfectly consistent with Ito's formula: if we take the limit in @eq-itonew2, replacing the limit of $(\Delta B_{t_i})^2$ with $(\d  B)^2 = \d  t$, we obtain @eq-itonew1.

To see the accuracy of Ito's approximation over different time steps, as well as the impact of the second-derivative term $\int_0^t (1/2)f''(B_s)\d s$, we encourage readers to interact with the plot below.  It examines the function $f(x)=\e^{x}$ (for which we have $f'(x)=\e^x$ and $f''(x) = \e^x$).  It simulates an approximate path of a Brownian motion as we have done before.  It then compares the true value of $\e^{B_{t_i}}$ to the Ito expansion 
$$\e^{B_t}=1 + \int_0^t \e^{B_s} \d B_s + \frac{1}{2}\int_0^t \e^{B_s} \d s$$
using the discretization 
$$\Delta \e^{B_{t_i}}= \e^{B_{t_{i-1}}} \Delta B_{t_i} + \frac{1}{2} \e^{B_{t_{i-1}}} \Delta t \,.$$
Notice that the discretization is just a second-order Taylor series expansion.  The discretization approximates the true value better if we take $n$ larger and $\Delta t$ smaller.  The important take-away from the figure is that the cumulative second-derivative terms in the discretization 
do not vanish as we take $n$ larger but instead continue to contribute significantly to the approximation.


::: {#fig-interactive_ito fig-cap="Accuracy of Ito Approximation"}
<iframe width="780" height="1000" src="https://derivatives-book-26ac36570fb8.herokuapp.com/math_finance_book_plots/ito_approx_plot"></iframe>
:::

## Functions of Time and a Brownian Motion

We extend the example in the previous section slightly.
Consider a process $Y$ defined as $Y_t = f(t, B_t)$ for some function $f$.  The following rule states that $Y$ is an Ito process with drift equal to
$$\frac{\partial f}{\partial t} + \frac{1}{2} \frac{\partial^2 f}{\partial B^2}$$ 
and diffusion coefficient equal to
$\partial f/\partial B$.
This is what we need to remember to make calculations.  The real meaning of $(\d B)^2$ is $\d t$, so we can (and will) substitute that in the following rule, but it may be easier to remember $(\d B)^2$.  This becomes more important when we consider more complex examples in the next sections.

::: Principle

If $f(t, B)$ is continuously differentiable in $t$ and twice continuously differentiable in $B$ and $Y_t= f(t, B_t)$ for a standard Brownian motion $B$, then
$$\d Y = \frac{\partial f(t, B)}{\partial t}\d t + \frac{\partial f(t, B)}{\partial B}\d B + \frac{1}{2} \frac{\partial^2 f(t, B)}{\partial B^2}(\d B)^2\,.
$$ {#eq-ito-firstformula}
:::

::: {.Example #example:ito1}
We can finish the discussion in @sec-s:discreteito regarding the process defined in @eq-ito_discrete_6 by applying @eq-ito-firstformula.  We want to show that the process satisfies @eq-ito_discrete_5.  We do that by differentiating and applying the @eq-ito-firstformula.  For $f(t, B) = \e^{(\mu - \sigma^2/2)t + \sigma B}$, we have
$$\frac{\partial f}{\partial t} = \left(\mu-\frac{1}{2}\sigma^2\right)f(t, B)\,,\quad \frac{\partial f}{\partial B} = \sigma f(t, B)\,, \quad \frac{\partial^2 f}{\partial B^2} = \sigma^2f(t, B)\,.$$
Therefore,
$$\d Y = \left(\mu-\frac{1}{2}\sigma^2\right)Y\d t + \sigma Y \d B + \frac{1}{2}\sigma^2 Y\,(\d B)^2 = \mu Y\d t + \sigma Y \d B$$
This verifies @eq-ito_discrete_5.
:::

## Functions of Time and an Ito Process

Now consider the more general case $Y_t = f(t, X_t)$ where $X$ is an Ito process.  As explained before, this means that
$$\d X_t = \alpha_t\d t + \theta_t \d B_t
$$ {#eq-itoagain}
for some stochastic processes $\alpha$ and $\theta$, where $B$ is a standard Brownian motion.  Then, from our previous rules, $(\d X)^2 = \theta^2\d t$.  Ito's formula in this more general case takes the same form as Calculation Rule \ref{ruleito1}, replacing the Brownian motion $B$ with the Ito process $X$.

::: Principle

If $f(t, X)$ is continuously differentiable in $t$ and twice continuously differentiable in $X$ and $Y_t= f(t, X_t)$ where $X$ is an Ito process, then
$$\d Y = \frac{\partial f(t, X)}{\partial t}\d t + \frac{\partial f(t, X)}{\partial X}\d X + \frac{1}{2} \frac{\partial^2 f(t, X)}{\partial X^2}(\d X)^2\,.
$$ {#eq-ito-secondformula}

:::

We can write @eq-ito-secondformula in terms of $\d t$ and $\d B$ terms by substituting from @eq-itoagain and using $(\d X)^2 = \theta^2\d t$.  This produces
$$\d Y = \left(\frac{\partial f(t, X)}{\partial t} + \alpha\frac{\partial f(t, X)}{\partial X} + \frac{1}{2}\theta^2 \frac{\partial^2 f(t, X)}{\partial X^2}\right)\d t + \theta\frac{\partial f(t, X)}{\partial X}\d B\,.$$

Here are some important examples of Ito's formula.

::: Rule
If $Y = X^\alpha$ for a constant $\alpha$, then
$$\d Y = \alpha X^{\alpha-1}\d X + \frac{1}{2}\alpha (\alpha - 1)X^{\alpha-2}(\d X)^2\,.$$
This is equivalent to 
$$\frac{\d Y}{Y} = \alpha \frac{\d X}{X} + \frac{\alpha(\alpha-1)}{2}\left(\frac{\d X}{X}\right)^2\,.
$$ {#ito-powerformula}
:::

::: Rule
If $Y=\mathrm{e}^X$, then 
$$\frac{\d  Y}{Y}=\d  X + \frac{(\d  X)^2}{2}\;.
$$ {#eq-rule5}
:::

::: Rule
If $Y=\log X$, then
$$
\d  Y=\frac{\d  X}{X} - \frac{1}{2}\left(\frac{\d  X}{X}\right)^2\;.
$$ {#eq-rule6}
:::

::: Example
We showed in the previous Example that $X$ defined in @eq-ito_discrete_6 satisfies @eq-ito_discrete_5, but it is also useful to see how we can start from @eq-ito_discrete_5 and deduce that @eq-ito_discrete_6 is the solution.  We can do that by taking logarithms.  Set $Y_t = \log X_t$.  Then, using @eq-rule6 and substituting from @eq-ito_discrete_5, we have
$$\d \log X = \left(\mu - \frac{1}{2}\sigma^2\right)\d t + \sigma\d B\,.$$
There is no $X$ on the right-hand side of this, so we can simply integrate to compute $\log X_t$ as
$$\log X_t = \log X_0 + \left(\mu - \frac{1}{2}\sigma^2\right)t + \sigma B_t\,.$$
Exponentiating gives @eq-ito_discrete_6.
:::

## Functions of Time and Multiple Ito Processes


Using @eq-products-differentials for products of differentials, we can state Ito's formula for a function of time and two Ito processes as follows.


::: Principle
If $Y_t = f(t, X_{1t}, X_{2t})$ where $X_1$ and $X_2$ are Ito processes and $f$ is continuously differentiable in $t$ and twice continuously differentiable in $X_1$ and $X_2$, then 
$$\begin{multline}
\d  Y =  \frac{\partial f}{\partial t}\d   t + \frac{\partial f}{\partial X_1}\d   X_1 +  \frac{\partial f}{\partial X_2}\d   X_2 \\+ \frac{1}{2} \frac{\partial^2 f}{\partial X_1^2}\,(\d  X_1)^2 +  \frac{1}{2}  \frac{\partial^2 f}{\partial X_2^2}\,(\d  X_2)^2 
+ \frac{\partial^2 f}{\partial X_1\partial X_2}\,(\d  X_1)(\d  X_2)\;.
\end{multline}$$ {#eq-itogeneralnew2}

:::

This is analogous to a second-order Taylor series expansion in the variables $X_1$ and $X_2$.  A similar formula applies to functions of more two Ito processes.  We just need to include a term for each $\d X_i$, each $(\d X_i)^2$ and each $(\d X_i)(\d X_j)$.

Here are some important examples.  We switch notation from $X_1$ and $X_2$ to $X$ and $Y$ and from $Y$ to $Z$ so we can drop the subscripts.  These formulas follow from @eq-itogeneralnew2 by taking $f(x,y)=xy$ or $f(x,y)=y/x$.

::: Rule
If $Z=XY$, then $\d  Z=X\d   Y+Y\d   X + (\d  X)(\d  Y)$.  We can write this as
$$
\frac{\d  Z}{Z}=\frac{\d  X}{X} + \frac{\d  Y}{Y} + \left(\frac{\d  X}{X}\right)\left(\frac{\d  Y}{Y}\right)\;.
$$ {#eq-rule2}
:::

::: Rule
If $Z=Y/X$, then 
$$\frac{\d  Z}{Z} = \frac{\d  Y}{Y} -\frac{\d  X}{X} - \left(\frac{\d  Y}{Y}\right)\left(\frac{\d  X}{X}\right) + \left(\frac{\d  X}{X}\right)^2\;.
$$ {#eq-rule4}
:::

The following is a special case of @eq-rule2 that we encounter often.

::: Rule
Let  
$$Y_t =\exp\left(\int_0^t q_s\d   s\right)$$
for some (possibly random) process $q$ and define $Z=XY$ for any Ito process $X$.
@eq-rule2 gives us
$$
\frac{\d  Z}{Z}=q\d   t + \frac{\d  X}{X}\;.
$$ {#eq-compdisc1}
This is the same as in the usual calculus.  
:::


## Exercises

::: {#exr-ito1}
Ito's Lemma can be used in different ways to get the same answer.  For example, let $X_t = a t + b B_t$ and use Ito's lemma on the function $e^{X_t}$.  Alternatively, let $f(t, B_t) = e^{a t + bB_t}$.  Use Ito's lemma on $f(,)$.
::: 


::: {#exr-ito2}
Let $\d X_t = \mu X_t \d t + \sigma X_t \d B_t$.  Use Ito's lemma to find $\log(X_t)$ .  What is the expected value and variance of $\log(X_t)$ ?
:::